In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from sklearn.linear_model import SGDClassifier

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from multiprocessing import Pool
# Function to load images and labels from a directory
import gc


def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        label = filename.split('.')[0]  # Extracting label from filename (assuming filename is in format "cat.xxx.jpg")
        if label == 'cat':
            label = 0
        elif label == 'dog':
            label = 1
        else:
            continue
        img = Image.open(os.path.join(folder, filename))
        images.append(img)
        labels.append(label)
    return images, labels

m = Xception(weights='imagenet', include_top=False, pooling='avg')
def extract_features(m,images):
    preprocessed_images = [image.resize((299, 299)) for image in images]
    preprocessed_images = [np.array(image) for image in preprocessed_images]
    preprocessed_images = np.array(preprocessed_images)
    preprocessed_images = preprocess_input(preprocessed_images)
    features = m.predict(preprocessed_images)
    del preprocessed_images  # Clear memory after feature extraction
    gc.collect()
    return features
# Path to the directory containing the images
folder_path = "train/train"

# Load images and labels
images, labels = load_images_from_folder(folder_path)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.8, random_state=42)
#feat = extract_features(X_train)
del images
del labels
gc.collect
# Initialize SVM classifier
clf = svm.SVC(kernel='linear')
batch_size = 1024
sgd_classifier = SGDClassifier(loss='log', alpha=0.0001, max_iter=1000, random_state=42)
# Train the SVM classifier batch-wise
num_samples = len(y_train)
num_batches = (num_samples + batch_size - 1) // batch_size

for batch_idx in range(num_batches):
    print(batch_idx+1)
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, num_samples)
    X_batch = extract_features(m,X_train[start_idx:end_idx])
    #X_batch = feat[start_idx:end_idx]
    y_batch = y_train[start_idx:end_idx]
    clf.fit(X_batch, y_batch)
    del X_batch
    del y_batch
    gc.collect()
# Train the SVM classifier
tnum_samples = len(y_test)
tnum_batches = (tnum_samples + batch_size - 1) // batch_size
y_pred=[]
for batch_idx in range(tnum_batches):
    print(batch_idx+1)
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, tnum_samples)
    x = extract_features(m,X_test[start_idx:end_idx])
    # Make predictions on the test data
    y_pred.append(clf.predict(x))
    del x
    gc.collect
# Concatenate arrays into a single array
y_pred = np.concatenate(y_pred)

# Convert the concatenated array to a Python list
y_pred = y_pred.tolist()
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

83683744/83683744 [==============================] - 0s 0us/step
1
32/32 [==============================] - 14s 189ms/step
2
31/31 [==============================] - 10s 296ms/step
1
32/32 [==============================] - 6s 203ms/step
2
32/32 [==============================] - 6s 204ms/step
3
32/32 [==============================] - 7s 208ms/step
4
32/32 [==============================] - 7s 212ms/step
5
32/32 [==============================] - 7s 215ms/step
6
32/32 [==============================] - 7s 219ms/step
7
32/32 [==============================] - 7s 223ms/step
8
26/26 [==============================] - 6s 227ms/step
Accuracy: 0.993


In [ ]:
img_path = 'path/to/sample.jpg'  # Replace 'sample.jpg' with the path to your image file
img = image.load_img(img_path, target_size=(299, 299))  # Xception model requires input shape of (299, 299)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Predict class probabilities
preds = m.predict(x)
a = clf.predict(preds)
print(a[0]) # 0 = cat while 1 = dog

1/1 [==============================] - 0s 32ms/step
0
